In [1]:
!pip install requests transformers pandas datetime vaderSentiment requests yfinance finnhub-python
!pip install nltk
!pip install vaderSentiment
!pip install yahoo-fin
!pip install finvizfinance
!pip install "websockets>=11.0"
!pip install yfinance yahoo_fin nltk textblob matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.2 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=f65c86fb177a05b784623029141960c954acd8a78bc427e2431564ef2b66f8c5
  Store

In [6]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz
from yahoo_fin import news as yahoo_fin_news
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Setup
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()
tz = pytz.timezone('US/Eastern')
now = datetime.now(tz)
start_window = now - timedelta(days=90)

TICKERS = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "NVDA", "PFE", "MRNA", "DIS", "T", "RYDE", "META", "NFLX", "AVGO", "AMD", "ORCL", "INTC",
           "QCOM", "IBM", "CRM", "JPM", "BAC", "GS", "WFC", "XOM", "CVX", "COP", "SLB", "PSX", "JNJ", "LLY", "UNH", "ABBV", "BMY",
           "WMT", "COST", "HD", "MCD", "NKE", "ADBE", "SHOP", "TSM", "PLTR", "UBER", "ABT", "ACN", "AIG", "AMGN", "AMT", "AXP", "BA", "BK", "BKNG",
           "BLK","BRK.B", "CAT", "CHTR", "CL", "CMCSA", "COF", "CVS", "DE", "DHR", "DUK", "EMR", "FDX", "GD", "GE", "GILD", "GM", "GOOGL", "HON",
           "INTU", "ISRG", "KO", "LIN", "LMT", "LOW", "MA", "MDLZ", "MDT", "MET", "MMM", "MO", "MRK", "NEE", "NOW", "PEP", "PG", "PM", "PYPL", "RTX",
           "SBUX", "SCHW", "SO", "SPG", "TGT", "TMO", "TMUS", "TXN", "UNP", "UPS", "USB", "V", "VZ", "XEL", "ABNB", "AKAM", "ALB", "ALGN",
           "ALLE", "ALL", "LNT", "ALLY","AON", "APD", "APH", "ADM", "ANET", "ADP", "APA", "APTV", "ARE", "AZO","AVB", "AVY", "BLL", "BBY", "BDX", "BR",
           "BF.B", "BIIB", "BWA", "BXP", "BSX", "COO", "CF", "CDNS", "CME", "CMS", "CCI", "CBRE", "CDW", "CERN", "CHD", "CI", "CINF", "CTAS", "CSX", "CTSH",
           "CLX", "CNC", "CNP", "CTRA", "CTVA", "COTY", "CPB", "COG", "KHC", "KR", "DLR", "DAL", "DRI", "DVA", "DXC", "ETN", "EOG", "EFX", "EQIX", "EQR",
           "ESS", "EL", "ETR", "EXC", "EXPE", "EXPD", "F", "FAST", "FIS", "FRC", "FLIR", "FLS", "FTNT", "GPS", "GRMN", "HCA", "HSY", "HUM", "IDXX", "IFF",
           "ILMN", "INCY", "IPG", "IQV", "IRM", "IT", "ITW", "JKHY", "JNPR", "K", "KEY", "KLAC", "KMB", "KMI", "KSS", "LHX", "LKQ", "LNC", "LRCX", "LW", "LVS",
           "MAR", "MAS", "MKC", "MCO", "MSI", "MSCI", "MS", "MTB", "MNST", "MRO", "MPC", "NDAQ", "NCLH", "NEM", "NLOK", "NTRS", "ORLY", "OXY", "PAYX", "PNR",
           "PNW", "PPG", "PPL", "PFG", "PGR", "PRU", "PXD", "QRVO", "REG", "REGN", "ROST", "RSG", "RMD", "SBAC", "SGEN", "SIVB", "SNA", "SWKS", "STT", "SYK",
           "TROW", "TRV", "TSCO", "UAL", "UDR", "VLO", "VRSK", "VRTX", "WAB", "WELL", "WHR", "WM", "WEC", "ZBRA"]

INTERVALS = {
    "1 Hour Ago": timedelta(hours=1),
    "4 Hours Ago": timedelta(hours=4),
    "Previous Day Close": timedelta(days=1),
    "7 Days Ago": timedelta(days=7)
}

class NewsAnalyzer:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)

    def fetch_articles(self):
      try:
          all_articles = [
              {
                  "Ticker": self.ticker,
                  "Title": art["title"],
                  "Summary": art["summary"],
                  "URL": art["link"],
                  "Original Published Time": art["published"],
                  "Datetime": datetime.strptime(art["published"], "%a, %d %b %Y %H:%M:%S %z").astimezone(tz)
              }
              for art in yahoo_fin_news.get_yf_rss(self.ticker)
              if start_window <= datetime.strptime(art["published"], "%a, %d %b %Y %H:%M:%S %z").astimezone(tz) <= now
          ]
          # Sort by datetime (most recent first), and take top 5
          all_articles.sort(key=lambda x: x["Datetime"], reverse=True)
          return all_articles[:3]
      except Exception as e:
          print(f"Error fetching news for {self.ticker}: {e}")
          return []

    def get_price_near(self, timepoint):
        try:
            hist = self.stock.history(
                start=timepoint.strftime('%Y-%m-%d'),
                end=(timepoint + timedelta(days=1)).strftime('%Y-%m-%d'),
                interval='1h'
            )
            if not hist.empty:
                hist['Delta'] = abs(hist.index - timepoint)
                return round(hist.sort_values('Delta').iloc[0]['Close'], 2)
        except:
            return None
        return None

    def analyze_article(self, article):
        base_time = article["Datetime"]
        base_price = self.get_price_near(base_time)
        if base_price is None:
            return None

        prices = {}
        pct_changes = {}
        for label, delta in INTERVALS.items():
            time_check = base_time - delta
            past_price = self.get_price_near(time_check)
            prices[label] = past_price if past_price else 'N/A'
            pct_changes[label + " % Change"] = (
                round(((base_price - past_price) / past_price) * 100, 2) if isinstance(past_price, (int, float)) else 'N/A'
            )

        return {
            **article,
            "News Age (Days)": (now - base_time).days,
            "News Time": base_time,
            "Current Price": base_price,
            **prices,
            **pct_changes,
            "Headline Sentiment": round(TextBlob(article["Title"]).sentiment.polarity, 2),
            "Summary Sentiment": round(vader.polarity_scores(article["Summary"])["compound"], 2)
        }

def run_full_analysis():
    full_dataset = []
    for ticker in TICKERS:
        analyzer = NewsAnalyzer(ticker)
        articles = analyzer.fetch_articles()
        for article in articles:
            result = analyzer.analyze_article(article)
            if result:
                full_dataset.append(result)

    df = pd.DataFrame(full_dataset)
    df.to_csv("refactored_yahoo_sentiment_7daytrack.csv", index=False)
    print("Saved to refactored_yahoo_sentiment_7daytrack.csv")
    return df

# Run it
df_result = run_full_analysis()
print(df_result.head())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-22 -> 2025-06-23)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-22 -> 2025-06-23)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-22 -> 2025-06-23)
ERROR:yfinance:$MSFT: possibly delisted; no price data found  (1h 2025-06-22 -> 2025-06-23)
ERROR:yfinance:$MSFT: possibly delisted; no price data found  (1h 2025-06-22 -> 2025-06-23)
ERROR:yfinance:$MSFT: possibly delisted; no price data found  (1h 2025-06-22 -> 2025-06-23)
ERROR:yfinance:$GOOG: possibly delisted; no price data found  (1h 2025-06-22 -> 2025-06-23)
ERROR:yfinance:$GOOG: possibly delisted; no price data found  (1h 2025-06-22 -> 2025-06-23)
ERROR:yfinance:$GOOG: possibly delisted; no price data found  (1h 2025-06-22 -> 2025-06-23)
ERROR:yfinance:$AMZN: possibly delisted; no p

Saved to refactored_yahoo_sentiment_7daytrack.csv
  Ticker                                              Title  \
0   AAPL  Steve Jobs Believed Teamwork Required 'Bumping...   
1   AAPL  Court filings reveal OpenAI and io’s early wor...   
2   AAPL  Is Apple Looking To Catch Up In AI With Big Deal?   
3   MSFT  Microsoft (MSFT) Surpasses Market Returns: Som...   
4   MSFT  How Microsoft Built a Reliable Dividend Track ...   

                                             Summary  \
0  Steve Jobs once likened a high-performing team...   
1  Legal filings submitted earlier this month fro...   
2  Apple’s slow response to the AI megatrend has ...   
3  Microsoft (MSFT) closed the most recent tradin...   
4  Microsoft Corporation (NASDAQ:MSFT) is one of ...   

                                                 URL  \
0  https://finance.yahoo.com/news/steve-jobs-beli...   
1  https://finance.yahoo.com/news/court-filings-r...   
2  https://finance.yahoo.com/m/036eb1b4-7079-3501...   
3  https:/

In [7]:
from google.colab import files
files.download("refactored_yahoo_sentiment_7daytrack.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>